This script reads and processes the recommendations and games csv. Recommendations are only kept if a same user gave more than 1000 views and the games are filtered to be before 2017 to match our feature data. The output file name is data_cleaned.csv, and is about  big.

Modify user id in a separate script named "userid.py".

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#read recommendations csv, the largest file
rec = pd.read_csv('recommendations.csv')
rec.head()

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,51580,0
1,304390,4,0,2017-02-17,False,11.5,2586,1
2,1085660,2,0,2019-11-17,True,336.5,253880,2
3,703080,0,0,2022-09-23,True,27.4,259432,3
4,526870,0,0,2021-01-10,True,7.9,23869,4


In [15]:
#read games csv
games = pd.read_csv('games.csv')
games.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


In [16]:
# combine the two datasets 
df_combined = pd.merge(rec, games[['app_id', 'date_release','title']], on='app_id', how='right')
df_combined.head()

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,date_release,title
0,13500,2.0,0.0,2015-05-30,True,0.4,3369378.0,14254690.0,2008-11-21,Prince of Persia: Warrior Within™
1,13500,4.0,0.0,2014-07-25,False,15.2,11294916.0,14254698.0,2008-11-21,Prince of Persia: Warrior Within™
2,13500,0.0,0.0,2021-12-15,True,19.4,11708578.0,14254708.0,2008-11-21,Prince of Persia: Warrior Within™
3,13500,13.0,0.0,2022-08-11,True,10.8,4177110.0,14254726.0,2008-11-21,Prince of Persia: Warrior Within™
4,13500,0.0,0.0,2021-05-18,False,10.9,8472871.0,14254752.0,2008-11-21,Prince of Persia: Warrior Within™


In [17]:
# find users who gave more than 1000 views so that we have enough traning and testing data for our target users
df_wow = df_combined[df_combined['user_id'].map(df_combined['user_id'].value_counts()) > 1000]
print(len(df_wow))
df_wow.head()

98568


,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,date_release,title
75,13500,0.0,0.0,2021-03-29,True,0.1,748899.0,14259610.0,2008-11-21,Prince of Persia: Warrior Within™
357,13500,7.0,2.0,2018-12-29,True,168.3,11283041.0,14280677.0,2008-11-21,Prince of Persia: Warrior Within™
363,13500,8.0,3.0,2021-11-12,True,1.2,11553593.0,14281036.0,2008-11-21,Prince of Persia: Warrior Within™
517,13500,3.0,0.0,2020-09-28,False,1.2,12822973.0,14292477.0,2008-11-21,Prince of Persia: Warrior Within™
955,13500,0.0,0.0,2013-08-25,True,17.3,11681010.0,30033106.0,2008-11-21,Prince of Persia: Warrior Within™


In [18]:
# now we can pick all games before 2017 from this dataframe as our traning set, since our features don't go further than 2016
df_wow['date_release'] = pd.to_datetime(df_wow['date_release'])
df_wow.head()

# extract the year
df_wow['year'] = df_wow['date_release'].dt.year
train = df_wow[df_wow['year']<2017]
print(max(df_wow['year']))

# pick out games released before 2017
# this should reduce the data length by about half
# compared to the original, this data set is reduced from 41 million to 47279 rows.
print(len(train))
# train.head()

2023
47279


C:\Users\VoidST\AppData\Local\Temp\ipykernel_13672\832421041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wow['date_release'] = pd.to_datetime(df_wow['date_release'])
C:\Users\VoidST\AppData\Local\Temp\ipykernel_13672\832421041.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wow['year'] = df_wow['date_release'].dt.year


In [19]:
# convert to markdown for website
markdown_table = train.head().to_markdown(index=False)
print(markdown_table)

|   app_id |   helpful |   funny | date       | is_recommended   |   hours |          user_id |   review_id | date_release        | title                             |   year |
|---------:|----------:|--------:|:-----------|:-----------------|--------:|-----------------:|------------:|:--------------------|:----------------------------------|-------:|
|    13500 |         0 |       0 | 2021-03-29 | True             |     0.1 | 748899           | 1.42596e+07 | 2008-11-21 00:00:00 | Prince of Persia: Warrior Within™ |   2008 |
|    13500 |         7 |       2 | 2018-12-29 | True             |   168.3 |      1.1283e+07  | 1.42807e+07 | 2008-11-21 00:00:00 | Prince of Persia: Warrior Within™ |   2008 |
|    13500 |         8 |       3 | 2021-11-12 | True             |     1.2 |      1.15536e+07 | 1.4281e+07  | 2008-11-21 00:00:00 | Prince of Persia: Warrior Within™ |   2008 |
|    13500 |         3 |       0 | 2020-09-28 | False            |     1.2 |      1.2823e+07  | 1.42925e+07 | 2008-

In [20]:
df = pd.read_csv('features_cleaned.csv')
df['title'] = df['ResponseName']

# merge features to dataset
train_features = pd.merge(train, df, on='title', how='right')
train_features = train_features.dropna()
train_features.head()

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,date_release,title,...,GenreIsRPG,GenreIsSimulation,GenreIsEarlyAccess,GenreIsFreeToPlay,GenreIsSports,GenreIsRacing,GenreIsMassivelyMultiplayer,PriceInitial,After2014,Expensive
0,10.0,0.0,0.0,2018-06-29,True,29.8,5164055.0,6175499.0,2000-11-01,Counter-Strike,...,False,False,False,False,False,False,False,9.99,0,0
1,10.0,49.0,3.0,2018-10-02,True,1.2,11783475.0,10861235.0,2000-11-01,Counter-Strike,...,False,False,False,False,False,False,False,9.99,0,0
2,10.0,11.0,5.0,2018-03-11,True,1.2,5108326.0,11111036.0,2000-11-01,Counter-Strike,...,False,False,False,False,False,False,False,9.99,0,0
3,10.0,9.0,0.0,2012-01-24,True,0.5,9231134.0,14867778.0,2000-11-01,Counter-Strike,...,False,False,False,False,False,False,False,9.99,0,0
4,10.0,26.0,9.0,2018-06-28,True,665.8,5390510.0,17091753.0,2000-11-01,Counter-Strike,...,False,False,False,False,False,False,False,9.99,0,0


In [21]:
# turn T/F into 1 and 0
columns_to_modify = [
    'is_recommended',
    'IsFree', 
    'GenreIsNonGame', 
    'GenreIsIndie', 
    'GenreIsAction', 
    'GenreIsAdventure', 
    'GenreIsCasual', 
    'GenreIsStrategy', 
    'GenreIsRPG', 
    'GenreIsSimulation',
    'GenreIsEarlyAccess', 
    'GenreIsFreeToPlay', 
    'GenreIsSports', 
    'GenreIsRacing', 
    'GenreIsMassivelyMultiplayer'
    ]
train_features[columns_to_modify] = train_features[columns_to_modify].astype(int)
train_features['user_id'] = train_features['user_id'].astype(int)
train_features.head()

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,date_release,title,...,GenreIsRPG,GenreIsSimulation,GenreIsEarlyAccess,GenreIsFreeToPlay,GenreIsSports,GenreIsRacing,GenreIsMassivelyMultiplayer,PriceInitial,After2014,Expensive
0,10.0,0.0,0.0,2018-06-29,1,29.8,5164055,6175499.0,2000-11-01,Counter-Strike,...,0,0,0,0,0,0,0,9.99,0,0
1,10.0,49.0,3.0,2018-10-02,1,1.2,11783475,10861235.0,2000-11-01,Counter-Strike,...,0,0,0,0,0,0,0,9.99,0,0
2,10.0,11.0,5.0,2018-03-11,1,1.2,5108326,11111036.0,2000-11-01,Counter-Strike,...,0,0,0,0,0,0,0,9.99,0,0
3,10.0,9.0,0.0,2012-01-24,1,0.5,9231134,14867778.0,2000-11-01,Counter-Strike,...,0,0,0,0,0,0,0,9.99,0,0
4,10.0,26.0,9.0,2018-06-28,1,665.8,5390510,17091753.0,2000-11-01,Counter-Strike,...,0,0,0,0,0,0,0,9.99,0,0


In [22]:
# get a list of IDs from this new dataframe
ids = train_features['user_id'].value_counts()

In [23]:
# export cleaned data
train_features.to_csv('data_cleaned.csv', index=False)
# export list of users (who are mad gamers and reviewed more than 1000 games) to choose from
ids.to_csv('list_of_users.csv')
# export a file including games after 2017 in case we want to predict some games for fun
df_wow.to_csv('comprehensive_user_data.csv', index=False)